In [1]:
import pandas as pd
import json
import os
import numpy as np
import re #regex 
import preprocessor as p
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
dfs = []
for r, d, f in os.walk('input'):
    for file in f:
        if '01_withheldtweets.json' in file or '01_plus_one_control.json' in file:  # alt: if ‘control' in file:
            dfs.append(pd.read_json('%s/%s' % (r, file), lines=True))

df_cen = pd.concat(dfs)
#df_cen = df_cen.dropna(subset=['withheld_in_countries'])

In [3]:
worthKeeping = ["text", "truncated", "user",
                "withheld_in_countries", "entities", "lang",
                "possibly_sensitive", "extended_tweet"]
df_cen = df_cen[worthKeeping]

In [4]:
df_cen

,text,truncated,user,withheld_in_countries,entities,lang,possibly_sensitive,extended_tweet
0,"On 30 Jan, a dozen embassies, including the US...",False,"{'id': 838404327693828097, 'id_str': '83840432...",NaN,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",en,0.0,NaN
1,کشتار، غارت وویرانی\n#بهمن_خانمانسوز,False,"{'id': 1284962399729209345, 'id_str': '1284962...",NaN,"{'hashtags': [{'text': 'بهمن_خانمانسوز', 'indi...",fa,NaN,NaN
2,EXACTLY,False,"{'id': 1343738986192367618, 'id_str': '1343738...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,NaN,NaN
3,Incroyable 😭😭😭😭😭😭😭,False,"{'id': 298429862, 'id_str': '298429862', 'name...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",fr,NaN,NaN
4,"@BJP4India, @INCIndia @samajwadiparty @Mayawati",False,"{'id': 3152843744, 'id_str': '3152843744', 'na...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",und,NaN,NaN
...,...,...,...,...,...,...,...,...
351,RT @jisoorubyjane: 💐 https://t.co/eZ8BhacVOl,False,"{'id': 1323962775450759169, 'id_str': '1323962...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",und,0.0,NaN
352,RT @withyouMT_TH: 210224 เพื่อนของมาร์คแชร์ภาพ...,False,"{'id': 1077173408977178625, 'id_str': '1077173...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",th,NaN,NaN
353,少しなおした,False,"{'id': 267534597, 'id_str': '267534597', 'name...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",ja,NaN,NaN
354,كوبونات خصم قويه من سيـفي و متاجر اخري\n\nاجمل...,True,"{'id': 554445460, 'id_str': '554445460', 'name...",NaN,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",ar,NaN,{'full_text': 'كوبونات خصم قويه من سيـفي و متا...


In [5]:
#replace the NaN with coherent values to make further processing easier
df_cen['possibly_sensitive'] = df_cen['possibly_sensitive'].fillna(0.0)
df_cen

,text,truncated,user,withheld_in_countries,entities,lang,possibly_sensitive,extended_tweet
0,"On 30 Jan, a dozen embassies, including the US...",False,"{'id': 838404327693828097, 'id_str': '83840432...",NaN,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",en,0.0,NaN
1,کشتار، غارت وویرانی\n#بهمن_خانمانسوز,False,"{'id': 1284962399729209345, 'id_str': '1284962...",NaN,"{'hashtags': [{'text': 'بهمن_خانمانسوز', 'indi...",fa,0.0,NaN
2,EXACTLY,False,"{'id': 1343738986192367618, 'id_str': '1343738...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,0.0,NaN
3,Incroyable 😭😭😭😭😭😭😭,False,"{'id': 298429862, 'id_str': '298429862', 'name...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",fr,0.0,NaN
4,"@BJP4India, @INCIndia @samajwadiparty @Mayawati",False,"{'id': 3152843744, 'id_str': '3152843744', 'na...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",und,0.0,NaN
...,...,...,...,...,...,...,...,...
351,RT @jisoorubyjane: 💐 https://t.co/eZ8BhacVOl,False,"{'id': 1323962775450759169, 'id_str': '1323962...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",und,0.0,NaN
352,RT @withyouMT_TH: 210224 เพื่อนของมาร์คแชร์ภาพ...,False,"{'id': 1077173408977178625, 'id_str': '1077173...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",th,0.0,NaN
353,少しなおした,False,"{'id': 267534597, 'id_str': '267534597', 'name...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",ja,0.0,NaN
354,كوبونات خصم قويه من سيـفي و متاجر اخري\n\nاجمل...,True,"{'id': 554445460, 'id_str': '554445460', 'name...",NaN,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",ar,0.0,{'full_text': 'كوبونات خصم قويه من سيـفي و متا...


In [6]:
#recover the full text for truncated tweets

dfRaw = df_cen.values
for line in dfRaw:
    if not pd.isna(line[-1]):
        line[0] = line[-1]["full_text"]
        
    #remove urls from tweets
    #they are shortened anyway so we can't make use of them
    line[0] = re.sub(r'http\S+', '', line[0])
    
    #flatten retweets
    line[0] = re.sub(r'RT @\S+:', '', line[0])

dfRaw = np.delete(dfRaw, len(worthKeeping)-1, axis=1) #remove "extended_tweet"
worthKeeping.remove("extended_tweet")

dfRaw = np.delete(dfRaw, 1, axis=1) #remove "truncated"
worthKeeping.remove("truncated")

In [7]:
#extract hashtags seperately

for line in dfRaw:
    line[3] = [x["text"] for x in line[3]["hashtags"]]
worthKeeping[3] = "hashtags"

In [8]:
#create a feature for user-verified and user-followers_count

print(dfRaw[0][1].keys())
verified = [line[1]["verified"] for line in dfRaw]
followers = [line[1]["followers_count"] for line in dfRaw]
user_id = [line[1]["id"] for line in dfRaw]

#for the location, Rebekah suggested to only spot the country name and discard the rest
listOfCountries = ['Afghanistan', 'Aland Islands', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia, Plurinational State of', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo', 'Congo, The Democratic Republic of the', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Heard Island and McDonald Islands', 'Holy See (Vatican City State)', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Republic of', 'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jersey', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', "Korea, Democratic People's Republic of", 'Korea, Republic of', 'Kuwait', 'Kyrgyzstan', "Lao People's Democratic Republic", 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao', 'Macedonia, Republic of', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Martinique', 'Mauritania', 'Mauritius', 'Mayotte', 'Mexico', 'Micronesia, Federated States of', 'Moldova, Republic of', 'Monaco', 'Mongolia', 'Montenegro', 'Montserrat', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'Niue', 'Norfolk Island', 'Northern Mariana Islands', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Palestinian Territory, Occupied', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Pitcairn', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Réunion', 'Romania', 'Russian Federation', 'Rwanda', 'Saint Barthélemy', 'Saint Helena, Ascension and Tristan da Cunha', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Martin (French part)', 'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten (Dutch part)', 'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Georgia and the South Sandwich Islands', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'South Sudan', 'Svalbard and Jan Mayen', 'Swaziland', 'Sweden', 'Switzerland', 'Syrian Arab Republic', 'Taiwan, Province of China', 'Tajikistan', 'Tanzania, United Republic of', 'Thailand', 'Timor-Leste', 'Togo', 'Tokelau', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'United States Minor Outlying Islands', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela, Bolivarian Republic of', 'Viet Nam', 'Virgin Islands, British', 'Virgin Islands, U.S.', 'Wallis and Futuna', 'Yemen', 'Zambia', 'Zimbabwe']
def findCountry(x):
    for country in listOfCountries:
        if x and country in x:
            return country
    return None

location = [findCountry(line[1]["location"]) for line in dfRaw]

dfRaw = np.c_[dfRaw, verified, followers, location, user_id]
worthKeeping += ["verified_account", "followers_count", "location", "user_id"]

dict_keys(['id', 'id_str', 'name', 'screen_name', 'location', 'url', 'description', 'translator_type', 'protected', 'verified', 'followers_count', 'friends_count', 'listed_count', 'favourites_count', 'statuses_count', 'created_at', 'utc_offset', 'time_zone', 'geo_enabled', 'lang', 'contributors_enabled', 'is_translator', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_background_tile', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color', 'profile_use_background_image', 'profile_image_url', 'profile_image_url_https', 'profile_banner_url', 'default_profile', 'default_profile_image', 'following', 'follow_request_sent', 'notifications'])


In [9]:
#binary feature for whether the tweet has been withheld anywhere

withheld = []
for line in dfRaw:
    if not isinstance(line[2], list):
        line[2] = []
    withheld.append(len(line[2]) != 0)
        
dfRaw = np.c_[dfRaw, withheld]
worthKeeping += ["withheld_anywhere"]

In [10]:
#sentiment analysis
#https://www.analyticsvidhya.com/blog/2022/07/sentiment-analysis-using-python/? with VADER


sentiment = SentimentIntensityAnalyzer()
#we made the assumption that sentiment analysis for this analyzer only works for english
res = np.array([[x for x in sentiment.polarity_scores(line[0]).values()] if line[4] == "en" else [0.0, 0.0, 0.0, 0.0] for line in dfRaw])

dfRaw = np.c_[dfRaw, res]
worthKeeping += ["neg", "neu", "pos", "compound"]

In [11]:
#popularity feature:
#build a score based on the values of followers_count, favourites_count, statuses_count
#compute a score from 0 to 1 for each, with (x - min)/(max - min), then comptute the average of these scores 

followers_count = np.array([line[1]["followers_count"] for line in dfRaw])
favourites_count = np.array([line[1]["favourites_count"] for line in dfRaw])
statuses_count = np.array([line[1]["statuses_count"] for line in dfRaw])

def normalize(array):
    return (array - np.min(array)) / (np.max(array) - np.min(array))

score = (1/3) * (normalize(followers_count) + normalize(favourites_count) + normalize(statuses_count))
dfRaw = np.c_[dfRaw, score]
worthKeeping += ["popularity_score"]

In [12]:
#reassemble the data in a pandas dataframe
df_cen = pd.DataFrame(dfRaw, columns = worthKeeping)
df_cen

,text,user,withheld_in_countries,hashtags,lang,possibly_sensitive,verified_account,followers_count,location,user_id,withheld_anywhere,neg,neu,pos,compound,popularity_score
0,"On 30 Jan, a dozen embassies, including the US...","{'id': 838404327693828097, 'id_str': '83840432...",[],[],en,0.0,True,16064,None,838404327693828097,False,0.0,1.0,0.0,0.0,0.011504
1,کشتار، غارت وویرانی\n#بهمن_خانمانسوز,"{'id': 1284962399729209345, 'id_str': '1284962...",[],[بهمن_خانمانسوز],fa,0.0,False,63,None,1284962399729209345,False,0.0,0.0,0.0,0.0,0.000753
2,EXACTLY,"{'id': 1343738986192367618, 'id_str': '1343738...",[],[],en,0.0,False,140,None,1343738986192367618,False,0.0,1.0,0.0,0.0,0.005414
3,Incroyable 😭😭😭😭😭😭😭,"{'id': 298429862, 'id_str': '298429862', 'name...",[],[],fr,0.0,False,486,None,298429862,False,0.0,0.0,0.0,0.0,0.035021
4,"@BJP4India, @INCIndia @samajwadiparty @Mayawati","{'id': 3152843744, 'id_str': '3152843744', 'na...",[],[],und,0.0,False,73,None,3152843744,False,0.0,0.0,0.0,0.0,0.000937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2047,💐,"{'id': 1323962775450759169, 'id_str': '1323962...",[],[],und,0.0,False,1045,None,1323962775450759169,False,0.0,0.0,0.0,0.0,0.009873
2048,210224 เพื่อนของมาร์คแชร์ภาพบน IG \n\n“เมื่อก...,"{'id': 1077173408977178625, 'id_str': '1077173...",[],[],th,0.0,False,31,None,1077173408977178625,False,0.0,0.0,0.0,0.0,0.066156
2049,少しなおした,"{'id': 267534597, 'id_str': '267534597', 'name...",[],[],ja,0.0,False,1486,None,267534597,False,0.0,0.0,0.0,0.0,0.024825
2050,كوبونات خصم قويه من سيـفي و متاجر اخري\n\nاجمل...,"{'id': 554445460, 'id_str': '554445460', 'name...",[],[],ar,0.0,False,2,None,554445460,False,0.0,0.0,0.0,0.0,0.000035


In [13]:
cleanCols = filter(lambda x: x != "user", worthKeeping)
df_clean = df_cen[cleanCols]

In [14]:
df_clean

,text,withheld_in_countries,hashtags,lang,possibly_sensitive,verified_account,followers_count,location,user_id,withheld_anywhere,neg,neu,pos,compound,popularity_score
0,"On 30 Jan, a dozen embassies, including the US...",[],[],en,0.0,True,16064,None,838404327693828097,False,0.0,1.0,0.0,0.0,0.011504
1,کشتار، غارت وویرانی\n#بهمن_خانمانسوز,[],[بهمن_خانمانسوز],fa,0.0,False,63,None,1284962399729209345,False,0.0,0.0,0.0,0.0,0.000753
2,EXACTLY,[],[],en,0.0,False,140,None,1343738986192367618,False,0.0,1.0,0.0,0.0,0.005414
3,Incroyable 😭😭😭😭😭😭😭,[],[],fr,0.0,False,486,None,298429862,False,0.0,0.0,0.0,0.0,0.035021
4,"@BJP4India, @INCIndia @samajwadiparty @Mayawati",[],[],und,0.0,False,73,None,3152843744,False,0.0,0.0,0.0,0.0,0.000937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2047,💐,[],[],und,0.0,False,1045,None,1323962775450759169,False,0.0,0.0,0.0,0.0,0.009873
2048,210224 เพื่อนของมาร์คแชร์ภาพบน IG \n\n“เมื่อก...,[],[],th,0.0,False,31,None,1077173408977178625,False,0.0,0.0,0.0,0.0,0.066156
2049,少しなおした,[],[],ja,0.0,False,1486,None,267534597,False,0.0,0.0,0.0,0.0,0.024825
2050,كوبونات خصم قويه من سيـفي و متاجر اخري\n\nاجمل...,[],[],ar,0.0,False,2,None,554445460,False,0.0,0.0,0.0,0.0,0.000035


In [15]:
df_clean["withheld_anywhere"].describe()

count     2052
unique       2
top       True
freq      1220
Name: withheld_anywhere, dtype: object

In [16]:
df_clean["neg"].describe()

count     2052.0
unique     155.0
top          0.0
freq      1724.0
Name: neg, dtype: float64

In [17]:
df_clean["popularity_score"].describe()

count     2052.000000
unique    1599.000000
top          0.000018
freq         3.000000
Name: popularity_score, dtype: float64

In [18]:
# Library: https://pypi.org/project/tweet-preprocessor/

# p.clean(file{.JSON, .txt}) or p.tokenize(file{.JSON, .txt}) or p.parse(file{.JSON, .txt})

# p.set_options(p.OPT.?)

In [19]:
# Option Name, Option Short Code:

# URL, p.OPT.URL

# Mention, p.OPT.MENTION

# Hashtag, p.OPT.HASHTAG 

# Reserved Words, p.OPT.RESERVED

# Emoji, p.OPT.EMOJI -> keeping for now

# Smiley, p.OPT.SMILEY -> keeping for now

# Number, p.OPT.NUMBER

In [20]:
p.set_options(p.OPT.URL, p.OPT.NUMBER, p.OPT.URL, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.HASHTAG)
df_processed = df_clean.copy()
df_processed["text"] = df_clean.apply({"text": lambda line: p.clean(line)}) 
df_processed

,text,withheld_in_countries,hashtags,lang,possibly_sensitive,verified_account,followers_count,location,user_id,withheld_anywhere,neg,neu,pos,compound,popularity_score
0,"On Jan, a dozen embassies, including the US an...",[],[],en,0.0,True,16064,None,838404327693828097,False,0.0,1.0,0.0,0.0,0.011504
1,کشتار، غارت وویرانی,[],[بهمن_خانمانسوز],fa,0.0,False,63,None,1284962399729209345,False,0.0,0.0,0.0,0.0,0.000753
2,EXACTLY,[],[],en,0.0,False,140,None,1343738986192367618,False,0.0,1.0,0.0,0.0,0.005414
3,Incroyable 😭😭😭😭😭😭😭,[],[],fr,0.0,False,486,None,298429862,False,0.0,0.0,0.0,0.0,0.035021
4,",",[],[],und,0.0,False,73,None,3152843744,False,0.0,0.0,0.0,0.0,0.000937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2047,💐,[],[],und,0.0,False,1045,None,1323962775450759169,False,0.0,0.0,0.0,0.0,0.009873
2048,เพื่อนของมาร์คแชร์ภาพบน IG “เมื่อก่อนทุกๆปีผมก...,[],[],th,0.0,False,31,None,1077173408977178625,False,0.0,0.0,0.0,0.0,0.066156
2049,少しなおした,[],[],ja,0.0,False,1486,None,267534597,False,0.0,0.0,0.0,0.0,0.024825
2050,"كوبونات خصم قويه من سيـفي و متاجر اخري اجمل "" ...",[],[],ar,0.0,False,2,None,554445460,False,0.0,0.0,0.0,0.0,0.000035


In [21]:
df_noduplicates = df_processed.copy()
df_noduplicates['withheld_in_countries'] = df_processed['withheld_in_countries'].astype(str)
df_noduplicates['hashtags'] = df_processed['hashtags'].astype(str)
df_noduplicates.drop_duplicates()
df_noduplicates

,text,withheld_in_countries,hashtags,lang,possibly_sensitive,verified_account,followers_count,location,user_id,withheld_anywhere,neg,neu,pos,compound,popularity_score
0,"On Jan, a dozen embassies, including the US an...",[],[],en,0.0,True,16064,None,838404327693828097,False,0.0,1.0,0.0,0.0,0.011504
1,کشتار، غارت وویرانی,[],['بهمن_خانمانسوز'],fa,0.0,False,63,None,1284962399729209345,False,0.0,0.0,0.0,0.0,0.000753
2,EXACTLY,[],[],en,0.0,False,140,None,1343738986192367618,False,0.0,1.0,0.0,0.0,0.005414
3,Incroyable 😭😭😭😭😭😭😭,[],[],fr,0.0,False,486,None,298429862,False,0.0,0.0,0.0,0.0,0.035021
4,",",[],[],und,0.0,False,73,None,3152843744,False,0.0,0.0,0.0,0.0,0.000937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2047,💐,[],[],und,0.0,False,1045,None,1323962775450759169,False,0.0,0.0,0.0,0.0,0.009873
2048,เพื่อนของมาร์คแชร์ภาพบน IG “เมื่อก่อนทุกๆปีผมก...,[],[],th,0.0,False,31,None,1077173408977178625,False,0.0,0.0,0.0,0.0,0.066156
2049,少しなおした,[],[],ja,0.0,False,1486,None,267534597,False,0.0,0.0,0.0,0.0,0.024825
2050,"كوبونات خصم قويه من سيـفي و متاجر اخري اجمل "" ...",[],[],ar,0.0,False,2,None,554445460,False,0.0,0.0,0.0,0.0,0.000035


In [31]:
n = 10
df_noduplicates['lang'].value_counts()[:n].index.tolist()

['en', 'tr', 'ur', 'und', 'ja', 'es', 'th', 'pt', 'ar', 'in']

In [23]:
df_english = df_noduplicates[df_noduplicates['lang'] == "en"] 
df_english

,text,withheld_in_countries,hashtags,lang,possibly_sensitive,verified_account,followers_count,location,user_id,withheld_anywhere,neg,neu,pos,compound,popularity_score
0,"On Jan, a dozen embassies, including the US an...",[],[],en,0.0,True,16064,None,838404327693828097,False,0.0,1.0,0.0,0.0,0.011504
2,EXACTLY,[],[],en,0.0,False,140,None,1343738986192367618,False,0.0,1.0,0.0,0.0,0.005414
6,THANK YOU DOYOUNG 💙,[],"['HAPPYDOYOUNGDAY', 'OurPreciousDoyoungDay', '...",en,0.0,False,161,Indonesia,1255688072160702464,False,0.0,0.485,0.515,0.8143,0.004411
8,If you would like the fellowship code please s...,[],[],en,0.0,False,702,None,2474129222,False,0.0,0.805,0.195,0.6239,0.00132
14,Cha Minho who is working part-time because he ...,[],[],en,0.0,False,574,None,1032588942,False,0.072,0.928,0.0,-0.1027,0.148693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2035,And that’s why Diana’s favorite son still has ...,[],[],en,0.0,False,576,None,1000076512534695937,False,0.0,0.75,0.25,0.4588,0.00595
2036,I already made pics,[],[],en,0.0,False,184,None,1253234646797729792,False,0.0,1.0,0.0,0.0,0.032786
2039,laur what is this ur already dating me????????...,[],[],en,0.0,False,19893,None,749745496948781056,False,0.0,1.0,0.0,0.0,0.018026
2040,Reminder: stop confusing or conflating the own...,[],[],en,0.0,False,464,None,867195180,False,0.162,0.672,0.166,0.2732,0.036454


In [24]:
df_turkish = df_noduplicates[df_noduplicates['lang'] == "tr"] 
df_turkish

,text,withheld_in_countries,hashtags,lang,possibly_sensitive,verified_account,followers_count,location,user_id,withheld_anywhere,neg,neu,pos,compound,popularity_score
9,DUYURU: Eylem yeri Beyazıt olarak değiştirilmi...,[],[],tr,0.0,False,1648,None,306078602,False,0.0,0.0,0.0,0.0,0.023433
59,adalet bakanı çözünce diktatör diyorsunuz,[],[],tr,0.0,False,115,None,1283419951,False,0.0,0.0,0.0,0.0,0.00164
98,Cezaevlerinde yüzlerce ağır hasta var.Bu hasta...,[],[],tr,0.0,False,2013,None,1212156582600929280,False,0.0,0.0,0.0,0.0,0.02643
122,Eğer devlet isterse kendine nasıl düşman oluşt...,[],"['Susmayacağız', 'AsağıyaBakmayacağız']",tr,0.0,False,1414,None,1244039305095503877,False,0.0,0.0,0.0,0.0,0.008421
152,kafam mı gğzel yoksa sen mi çok güzelsiö,[],[],tr,0.0,False,2048,None,1318686026282532871,False,0.0,0.0,0.0,0.0,0.004629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1770,AFJKJJHKKKKKKKDKKDTKKENIWNE8FN8EQM9QNEURKSKQKK...,[],[],tr,0.0,False,1232,None,1358613676954161155,False,0.0,0.0,0.0,0.0,0.008274
1794,bin atamayla milyondan fazla öğrencinin dersin...,[],[],tr,0.0,False,76,None,1290271765734862849,False,0.0,0.0,0.0,0.0,0.000542
1802,Kafayı yiyorum şuan biz bu ülkede ne sıfatla y...,[],"['ZKusağıSiziSandıktaBekliyor', 'ZKuşağıUnutmaz']",tr,0.0,False,262,None,1276602369955028994,False,0.0,0.0,0.0,0.0,0.004186
1838,"“İnce Memed”, “Orta Direk” gibi eserlerin sahi...",[],['YaşarKemal'],tr,0.0,False,6099,None,1211667497922940929,False,0.0,0.0,0.0,0.0,0.092191


In [25]:
df_urdu = df_noduplicates[df_noduplicates['lang'] == "ur"] 
df_urdu

,text,withheld_in_countries,hashtags,lang,possibly_sensitive,verified_account,followers_count,location,user_id,withheld_anywhere,neg,neu,pos,compound,popularity_score
287,The saddest picture of ... مسلمانو، ان صہیونیو...,['IN'],[],ur,0.0,False,1738,Pakistan,1232002966225334273,True,0.0,0.0,0.0,0.0,0.002956
288,The saddest picture of ... مسلمانو، ان صہیونیو...,['IN'],[],ur,0.0,False,1738,Pakistan,1232002966225334273,True,0.0,0.0,0.0,0.0,0.002956
299,The saddest picture of ... مسلمانو، ان صہیونیو...,['IN'],[],ur,0.0,False,99,None,628781136,True,0.0,0.0,0.0,0.0,0.021336
300,The saddest picture of ... مسلمانو، ان صہیونیو...,['IN'],[],ur,0.0,False,99,None,628781136,True,0.0,0.0,0.0,0.0,0.021336
306,تاریخ اسلام میں پہلی مرتبہ حرمین شریفین کو اس ...,['IN'],[],ur,0.0,False,115,Pakistan,1165822950,True,0.0,0.0,0.0,0.0,0.012648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1687,🥀 *بیٹھنے سے پہلے تحیۃ المسجد!* 🔹 *رسول اللہ ص...,['IN'],[],ur,0.0,False,1624,None,1283912379038916613,True,0.0,0.0,0.0,0.0,0.009828
1691,رسول اللہ ﷺ نے فرمایا :’’ میں نے معراج کی رات ...,['IN'],[],ur,0.0,False,3901,None,1279264068650614784,True,0.0,0.0,0.0,0.0,0.031545
1692,اُٹھ بھی سکتا ہے کبھی میرا یقیں تجھ پر سے میں ...,['IN'],['ادب_کا_سفر'],ur,0.0,False,1624,None,1283912379038916613,True,0.0,0.0,0.0,0.0,0.009837
1693,ہم بھی تھے تیری طرح آوارہ​ کیا تجھے بادِ صبا ی...,['IN'],['ادب_کا_سفر'],ur,0.0,False,1624,None,1283912379038916613,True,0.0,0.0,0.0,0.0,0.009857


In [29]:
df_nolang = df_noduplicates[df_noduplicates['lang'] == "und"] 
df_nolang
df_nolang_withheld = df_nolang[df_nolang['withheld_anywhere'] == "True"]
df_nolang_withheld

,text,withheld_in_countries,hashtags,lang,possibly_sensitive,verified_account,followers_count,location,user_id,withheld_anywhere,neg,neu,pos,compound,popularity_score


In [27]:
df_japanese = df_noduplicates[df_noduplicates['lang'] == "ja"] 
df_japanese

,text,withheld_in_countries,hashtags,lang,possibly_sensitive,verified_account,followers_count,location,user_id,withheld_anywhere,neg,neu,pos,compound,popularity_score
10,その言葉一番嫌い,[],[],ja,0.0,False,439,None,1291156481656541184,False,0.0,0.0,0.0,0.0,0.002787
11,かわいい～～～～ฅ•ω•ฅﾆｬｰ,[],[],ja,0.0,False,48,None,980629746949042176,False,0.0,0.0,0.0,0.0,0.005541
20,これよこれ〜👀✨,[],[],ja,0.0,False,89,None,2263267994,False,0.0,0.0,0.0,0.0,0.005313
35,こんなテンションで絡むぞ！ っていう忠告のつもりで😆 ﾎｲﾎｲ₍₍ ◝('ω'◝) ⁾⁾ ₍...,[],[],ja,0.0,False,131,None,1354052144677101568,False,0.0,0.0,0.0,0.0,0.000358
45,上の方ですかね！,[],[],ja,0.0,False,2067,None,782954232295018498,False,0.0,0.0,0.0,0.0,0.026728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983,美美滴,[],[],ja,0.0,False,13,None,1070024907226984449,False,0.0,0.0,0.0,0.0,0.000899
1986,余韻すぎる 人で家でUnder Pressureずっと歌ってる,[],[],ja,0.0,False,130,None,1089466451528830976,False,0.0,0.0,0.0,0.0,0.002027
2045,ということで、今朝は朝イチから 「スーパー戦隊MOVIEレンジャー2021」 を観てきました...,[],[],ja,0.0,False,871,None,846209331615825920,False,0.0,0.0,0.0,0.0,0.157265
2049,少しなおした,[],[],ja,0.0,False,1486,None,267534597,False,0.0,0.0,0.0,0.0,0.024825


In [32]:
df_spanish = df_noduplicates[df_noduplicates['lang'] == "es"] 
df_spanish

,text,withheld_in_countries,hashtags,lang,possibly_sensitive,verified_account,followers_count,location,user_id,withheld_anywhere,neg,neu,pos,compound,popularity_score
54,"¡Somos , somos Pueblo!",[],['ComunEs'],es,0.0,False,4379,Colombia,4904010619,False,0.0,0.0,0.0,0.0,0.006922
55,un masaje con velas y cosas exóticas con Avril...,[],[],es,0.0,False,1,None,1080913191566225408,False,0.0,0.0,0.0,0.0,0.000002
70,de qué equipo erais cuando os aburríais en cla...,[],[],es,0.0,False,88,None,712555238,False,0.0,0.0,0.0,0.0,0.047107
87,el humor se termina cuando se trata d una pers...,[],[],es,0.0,False,71,None,1297636727729926145,False,0.0,0.0,0.0,0.0,0.001763
97,EL MUGRIENTO CAFIERITO....👇👇👇👇👇👇👇👇,[],[],es,0.0,False,2268,Argentina,759039470406541315,False,0.0,0.0,0.0,0.0,0.095215
132,"Con gusto se programa atención, quedamos a la ...",[],[],es,0.0,False,13523,None,75343587,False,0.0,0.0,0.0,0.0,0.012607
159,donde consigo una tia adriana?,[],[],es,1.0,False,163,Argentina,4816350315,False,0.0,0.0,0.0,0.0,0.020244
221,Mi amor yo no soy mago pero quieres ver cómo t...,[],[],es,0.0,False,1131,None,1004485493936066560,False,0.0,0.0,0.0,0.0,0.033372
229,no se dímelo tu,[],[],es,0.0,False,51,None,1214243534653382658,False,0.0,0.0,0.0,0.0,0.000209
259,Alta pena JAJAJJAJAJAJAJAJJSJAJAJAJAJA🤣🤧,[],[],es,0.0,False,189,None,1306329429824745472,False,0.0,0.0,0.0,0.0,0.000769


In [40]:
df_thai = df_noduplicates[df_noduplicates['lang'] == "th"] 
df_thai.count()

text                     57
withheld_in_countries    57
hashtags                 57
lang                     57
possibly_sensitive       57
verified_account         57
followers_count          57
location                  6
user_id                  57
withheld_anywhere        57
neg                      57
neu                      57
pos                      57
compound                 57
popularity_score         57
dtype: int64

In [38]:
df_portuguese = df_noduplicates[df_noduplicates['lang'] == "pt"] 
df_portuguese.count()

text                     52
withheld_in_countries    52
hashtags                 52
lang                     52
possibly_sensitive       52
verified_account         52
followers_count          52
location                  3
user_id                  52
withheld_anywhere        52
neg                      52
neu                      52
pos                      52
compound                 52
popularity_score         52
dtype: int64

In [39]:
df_arabic = df_noduplicates[df_noduplicates['lang'] == "ar"] 
df_arabic.count()

text                     45
withheld_in_countries    45
hashtags                 45
lang                     45
possibly_sensitive       45
verified_account         45
followers_count          45
location                  7
user_id                  45
withheld_anywhere        45
neg                      45
neu                      45
pos                      45
compound                 45
popularity_score         45
dtype: int64

In [37]:
df_indian = df_noduplicates[df_noduplicates['lang'] == "in"] 
df_indian.count()

text                     36
withheld_in_countries    36
hashtags                 36
lang                     36
possibly_sensitive       36
verified_account         36
followers_count          36
location                  3
user_id                  36
withheld_anywhere        36
neg                      36
neu                      36
pos                      36
compound                 36
popularity_score         36
dtype: int64